In [1]:
from sef import read 
import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
# so we have a sampling rate of 1000
# If you just want to know what EEG state the person is in, you could use np.max instead of np.mean. 
# Then whichever band has the max will tell you about the person's state.

fs = 1000                               # Sampling rate (1000 Hz)

def get_eeg_bands(channel_data, display=False):
    band_to_data = {}
    # Get real amplitudes of FFT (only in postive frequencies)
    fft_vals = np.absolute(np.fft.rfft(channel_data))

    # Get frequencies for amplitudes in Hz
    fft_freq = np.fft.rfftfreq(len(channel_data), 1.0/fs)

    # Define EEG bands
    eeg_bands = {'Delta': (0, 4),
                 'Theta': (4, 8),
                 'Alpha': (8, 12),
                 'Beta': (12, 30),
                 'Gamma': (30, 45)}

    my_colors = ["g", "b","r","y","k"]

    # Take the mean of the fft amplitude for each EEG band
    eeg_band_fft = dict()
    for band in eeg_bands:  
        freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                           (fft_freq <= eeg_bands[band][1]))[0]
        eeg_band_fft[band] = np.mean(fft_vals[freq_ix])
        band_to_data[band] = channel_data[freq_ix]
    
    if display:
        # Plot the data (using pandas here cause it's easy)
        df = pd.DataFrame(columns=['band', 'val'])
        df['band'] = eeg_bands.keys()
        df['val'] = [eeg_band_fft[band] for band in eeg_bands]
        ax = df.plot.bar(x='band', y='val', legend=False, color=my_colors)
        ax.set_xlabel("EEG band")
        ax.set_ylabel("Mean band Amplitude")
    
    return band_to_data


def get_lengths(channel_to_bands):
    return {x:len(channel_to_bands[x]) for x in channel_to_bands}

def get_bands_to_matrix(sample_data):
    data = np.array(sample_data)
    #print(data.shape)

    # should split the sef of the samples to the bands 
    band_lengths = {}
    bands_to_matrix = {}

    for channel_index in range(data.shape[1]):
        band_to_data = get_eeg_bands(data[:,channel_index])
        band_lengths = get_lengths(band_to_data)
        for band in band_to_data:
            channel_data = np.array(band_to_data[band])
            channel_data = np.reshape(channel_data, (channel_data.shape[0],-1))

            if band not in bands_to_matrix:
                bands_to_matrix[band] = channel_data
            else:
                bands_to_matrix[band] = np.append(bands_to_matrix[band], channel_data, axis=1)

#     for band in bands_to_matrix:
#         print(bands_to_matrix[band].shape)
    return bands_to_matrix, band_lengths

In [3]:
import os

pd_dir = "D:/pd_data/band1_35Hz_PD_EC"
hc_dir = "D:/pd_data/band1_35Hz_HC_EC"

pd_files = [os.path.join(pd_dir,x) for x in os.listdir(pd_dir) if ".sef" == os.path.splitext(x)[1]]
hc_files = [os.path.join(hc_dir,x) for x in os.listdir(hc_dir) if ".sef" == os.path.splitext(x)[1]]

print(len(pd_files))
print(len(hc_files))

44
24


In [4]:
band_to_lengths = {}
band_to_matricies = {}

def create_comb(files, band_to_lengths, band_to_matrices):
    for filepath in files:
        print(filepath)
        sample_data = read(filepath)[2]
        bands_to_matrix, band_lengths = get_bands_to_matrix(sample_data)
        for band in band_lengths:
            if band not in band_to_lengths:
                band_to_lengths[band] = [band_lengths[band]]
                band_to_matricies[band] = bands_to_matrix[band]
            else:
                band_to_lengths[band].append(band_lengths[band])
                #print(band_to_matrices[band].shape)
                #print(bands_to_matrix[band].shape
                # add the columns
                band_to_matrices[band] = np.append(band_to_matrices[band], bands_to_matrix[band], axis=0)
                
create_comb(pd_files, band_to_lengths, band_to_matricies)
create_comb(hc_files, band_to_lengths, band_to_matricies)

D:/pd_data/band1_35Hz_PD_EC\EC_CT001_20110526_1514_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT002_20110531_1431_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT003_20110607_1528_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT007_20110705_1523_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT013_20111019_1405_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT014_20111114_1411_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT015_20111118_1418_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT016_20111116_1422_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT017_20120104_1234_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT018_20120109_1431_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT019_20120112_1352_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT022_20120131_1421_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT024_20120214_1412_Stitch_band_1_35.sef
D:/pd_data/band1_35Hz_PD_EC\EC_CT025_20120208_1419_Stitch_band_1

KeyError: 'theta'

In [15]:
from sef_eeg_microstates import *
import time 

start = time.time()
print band_to_lengths.keys()
print(band_to_matricies['Theta'].shape)

def generate_microstates(band_to_matrices):
    for band in band_to_matricies:
        print band
        data = band_to_matricies[band]
        print("%d MB" % (data.size * data.itemsize/1024/1024))
        
        # we need to get the 213 channels representing the 213 channels in y-axis
        from preprocessing import chan_ID_213 as chs
        n_maps = 4 # the number of classes 
        mode = ["aahc", "kmeans", "kmedoids", "pca", "ica"][1]
        print("Clustering algorithm: {:s}".format(mode))
        locs = []
        maps, microstates, gfp_peaks, gev = clustering(data, fs, chs, locs, mode, n_maps, doplot=False)
        print("The length of the sequence: {0}".format(len(microstates)))
        print("The number of peaks: {0}".format(len(gfp_peaks)))
        
        # save the peaks and the sequences 
        np.save("microstates_{0}_4".format(band), microstates)
        np.save("peaks_{0}_4".format(band), gfp_peaks)
        
        # need to save the 
generate_microstates(band_to_matricies)
print("Total time: {0}".format(time.time()-start))

['Theta', 'Beta', 'Delta', 'Gamma', 'Alpha']
(48404L, 213L)
Theta
78 MB
Clustering algorithm: kmeans
	[+] Data format for clustering [GFP peaks, channels]: 1401 x 213

	[+] Clustering algorithm: mod. K-MEANS.
The number of peaks: 1401
		K-means run 1/5 converged after 31 iterations.
		K-means run 2/5 converged after 14 iterations.
		K-means run 3/5 converged after 29 iterations.
		K-means run 4/5 converged after 17 iterations.
		K-means run 5/5 converged after 14 iterations.



(48404L, 213L)
(4L, 213L)



	[+] Computation time: 37.85

	[+] Global explained variance GEV = 0.593
		GEV_0: 0.090
		GEV_1: 0.212
		GEV_2: 0.099
		GEV_3: 0.192
The length of the sequence: 48404
The number of peaks: 1401
Beta
353 MB
Clustering algorithm: kmeans
	[+] Data format for clustering [GFP peaks, channels]: 6203 x 213

	[+] Clustering algorithm: mod. K-MEANS.
The number of peaks: 6203
		K-means run 1/5 converged after 46 iterations.
		K-means run 2/5 converged after 37 iterations.
		K-means run 3/5 conv

In [16]:
# get the total lengths as objects 
print band_to_lengths

{'Theta': [712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 706, 712, 712, 712, 712, 712, 712, 706, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712], 'Beta': [3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3176, 3203, 3203, 3203, 3203, 3203, 3203, 3176, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203, 3203], 'Delta': [712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 706, 712, 712, 712, 712, 712, 712, 706, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 712, 